In [67]:
import numpy as np
import fpsample
import open3d as o3d
from numba import njit, prange
import open3d.core as o3c


In [68]:
np.random.seed(42)
point_cloud = np.random.uniform(low=0.0, high=1.0, size=(1000, 3))
point_cloud.shape

(1000, 3)

In [69]:
#random subsampling
idx = np.random.choice(point_cloud.shape[0], size=100, replace=False)
random_samples = point_cloud[idx]
random_samples.shape

(100, 3)

In [70]:
#fps samples
kdline_fps_samples_idx = fpsample.bucket_fps_kdline_sampling(point_cloud, 100, h=3)
fps_samples = point_cloud[kdline_fps_samples_idx]
fps_samples.shape

(100, 3)

In [71]:
#simple FPS implementation
def FPS(point_cloud, k):
    start_pt = np.random.randint(0, point_cloud.shape[0])

    fps_indices = np.full(k, -1)
    fps_indices[0] = start_pt

    min_distances = np.full(point_cloud.shape[0], np.inf)

    for i in range(1, k):
        prev_pt = point_cloud[fps_indices[i-1]]
        dist = np.linalg.norm(point_cloud - prev_pt, axis=1)
        min_distances = np.minimum(min_distances, dist) 
        fps_indices[i] = np.argmax(min_distances)
    return fps_indices

In [72]:
def triangle_area(points):
    vec1 = points[1]-points[0]
    vec2 = points[2]-points[0]
    area = np.cross(vec1, vec2)
    return np.linalg.norm(area)*0.5

In [73]:
bunny = o3d.data.BunnyMesh()
mesh = o3d.io.read_triangle_mesh(bunny.path)
points = np.asarray(mesh.vertices)
faces = np.asarray(mesh.triangles)
points.shape, faces.shape

((35947, 3), (69451, 3))

In [74]:
faces_area = np.zeros(faces.shape[0])
for i in prange(faces.shape[0]):
    face = faces[i]
    vertices = points[face]
    area = triangle_area(vertices)
    faces_area[i] = area
sum_area = np.sum(faces_area)
faces_area = faces_area/ sum_area

In [75]:
pcd = mesh.sample_points_uniformly(number_of_points=100000)
o3d.visualization.draw_geometries([pcd])

In [76]:
bunny_points = np.asarray(pcd.points)
bunny_points.shape

(100000, 3)

In [77]:
fps_indices = FPS(bunny_points, 2048)
fps_samples = bunny_points[fps_indices]

pcd = o3d.t.geometry.PointCloud(np.array(fps_samples, dtype=np.float32))
o3d.visualization.draw_geometries([pcd.to_legacy()])

In [78]:
kdline_fps_samples_idx = fpsample.bucket_fps_kdline_sampling(bunny_points, 2048, h=9)
fps_samples = bunny_points[kdline_fps_samples_idx]
pcd = o3d.t.geometry.PointCloud(np.array(fps_samples, dtype=np.float32))
o3d.visualization.draw_geometries([pcd.to_legacy()])

In [79]:
#voxel grid downsampling
voxel_size = 0.025
v_points = np.floor(points/voxel_size)
print(v_points[0:5])
voxel_points, indices = np.unique(v_points, axis=0, return_index=True)

[[-2.  5.  0.]
 [-2.  5.  0.]
 [-3.  6.  1.]
 [-1.  5.  0.]
 [-1.  5.  0.]]


In [80]:
pcd = o3d.t.geometry.PointCloud(np.array(points[indices], dtype=np.float32))
o3d.visualization.draw_geometries([pcd.to_legacy()])